In [ ]:
from challenges_tools import challenges_data
from challenges_tools import champions_by_challenge
from challenges_tools import challenges_by_champions
from challenges_tools import find_comp
from challenges_tools import find_comp_randomly
from challenges_tools import champions_probability_table
from challenges_tools import plot_champion_distribution
from champions_roles import best_fit_roles

%load_ext autoreload
%autoreload 2

from collections import defaultdict
import json

In [ ]:
challenges = []
for k, v in challenges_data.items():
    challenges.extend(v)

In [ ]:
n_champs_challenges =  {challenge["id"] for challenge in challenges if challenge["qte"] == 5 and challenge["id"] != 303408}
print(n_champs_challenges)

In [ ]:
# find best comp by faction
n_champs_challenges_comps = defaultdict(list)

for n_champs_challenge in n_champs_challenges:
    c = champions_by_challenge[n_champs_challenge]
    comps = []
    for comp, comp_challenges in find_comp(c, threshold_min=2):
        comp_roles, stupidity_level, off_role = best_fit_roles(comp)
        comps.append((comp_roles, list(comp_challenges), stupidity_level))

    comps.sort(key=lambda l: (-len(l[1]), l[2]))

    n_champs_challenges_comps[n_champs_challenge] = comps

n_champs_challenges_comps = dict(n_champs_challenges_comps)

In [ ]:
for f, v in n_champs_challenges_comps.items():
    print(f, len(v))

In [ ]:
comp_found = set()
comps_ = defaultdict(list)

champions, p = champions_probability_table(power=4, alpha=0)
plot_champion_distribution(champions, p)

for comp, comp_challenges in find_comp_randomly(champions, p, max_depth=1e7):
    if len(comp_challenges) < 6:
        continue

    comp_roles, stupidity_level, off_role = best_fit_roles(comp)
    
    comp_str = str(sorted(list(comp)))
    if comp_str not in comp_found:
        comp_found.add(comp_str)
        key = f"Best compositions ({len(comp_challenges)} challenges)"
        comps_[key].append((comp_roles, list(comp_challenges), stupidity_level))

comps_ = dict(comps_)

In [ ]:
for k, v in comps_.items():
    print(k, len(v))

In [ ]:
json.dump(n_champs_challenges_comps | comps_, open("static/compositions.json", "w"))